In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, os.path
import re

%matplotlib inline

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [25]:
not_inc_regions = [ 
                    'армавир',
                    'амурская область',
                    'архангельсая область без ао',
                    'архангельская область',
                    'архангельская область без авт. округа',
                    'архангельская область без авт.округа',
                    'архангельская область без автономного округа',
                    'архангельская область без ао',
                    'архангельская область без нао',
                    'архангельская область кроме ненецкого автономного округа',
                    'астраханская область',
                    'белская область',
                    'в том числе: ненецкий автономный округ',
                    'владимирская область',
                    'вологда',
                    'дагестан',
                    'дальневосточный',
                    'еврейская авт.область',
                    'еврейская автономная область',
                    'забайкальский край',
                    'ингушетия',
                    'кабардино-балкарская',
                    'калмыкия',
                    'камчатский край',
                    'карачаево-черкесская',
                    'краснодар',
                    'крым',
                    'курганская область',
                    'магаданская область',
                    'марий эл',
                    'мурманская область',
                    'ненецкий авт.округ',
                    'ненецкий автономный округ',
                    'ненецкий автономный округ архангельская область',
                    'ненецкий ао',
                    'новороссийск',
                    'новская область',
                    'оренбургская область',
                    'отгружено товаров собственного производства, выполнено работ и услуг собственными силами малыми предприятиями включая микропредприятия',
                    'приволжский',
                    'псковская область',
                    'рязанская область',
                    'саха якутия',
                    'сахалинская область',
                    'севастополь',
                    'северная осетия - алания',
                    'северная осетия- алания',
                    'северная осетия-алания',
                    'северо-западный',
                    'северо-кавказский',
                    'сибирский',
                    'сочи',
                    'средняя численность работников малых предприятий без микропредприятий',
                    'средняя численность работников микропредприятий',
                    'тамбовская область',
                    'тверская область',
                    'томск',
                    'уральский',
                    'хабаровский край',
                    'хакасия',
                    'череповец',
                    'чеченская',
                    'чита',
                    'чувашская',
                    'чувашская - чувашия',
                    'чувашская -чувашия',
                    'чукотский авт.округ',
                    'чукотский автономный округ',
                    'южный',
                    'ямало-ненецкий авт.округ',
                    'ямало-ненецкий автономный округ',
                    'ямало-ненецкий автономный округ тюменская область',
                    'ямало-ненецкий ао',
                    'nan'
]

In [26]:
def replaces(data):
    map_repl = {'город федерального значения': '',
                'город': '',
                ')': '',
                '(': '',
                'h': 'н',
                'k': 'к',
                'республика': '',
                'г.': '',
               'столица российской федерации': ''}
    for old_value, new_value in map_repl.items():
        data['region'] = data['region'].apply(lambda x: str(x).lower().replace(old_value, new_value).strip())
    data['region'] = data['region'].apply(lambda x: re.sub(r'\s+', ' ', x))
    return data

In [27]:
def preprocess_data(data):
    for col in data.columns[1:]:
        data = replaces(data)
        data[col] = data[col].apply(lambda x: str(x).strip().replace('-', 'nan').replace(' ', '').replace(',', '.')).astype('float')
        data = data[data['region'].apply(lambda x: 'федерация' not in x and 'федеральный округ' not in x)]
        data = data.iloc[:-1]

        regions_for_repl = {
            'тюменская область без ао': 'тюменская область',
            'удмуртская': 'удмуртия',
            'ханты-мансийский ао-югра': 'ханты-мансийский ао',
            'кемеровская область - кузбасс': 'кемеровская область',
            'адыгея адыгея': 'адыгея',
            'тюменская область без автономных округов': 'тюменская область',
            'тюменская область без авт. округов': 'тюменская область',
            'белская область': 'белгородская область',
            'нижеская область': 'нижегородская область',
            'татарстан татарстан': 'татарстан',
            'тюменская область без авт.округов': 'тюменская область'
        }
        
        data.loc[data['region'].apply(lambda x: 'ханты' in x), 'region'] = 'ханты-мансийский ао'
        
        for old_region_name, new_region_name in regions_for_repl.items():
            data['region'] = data['region'].apply(lambda x: x.replace(old_region_name, new_region_name))
        
    return data

In [28]:
def merge_data(file_paths):
    for i, path in enumerate(file_paths):

        data = pd.read_csv(path, sep='\t', encoding='windows-1251')
        data = preprocess_data(data)
        
        if i == 0:
            merged_data = data.copy()
        elif i != 0:
            merged_data = pd.merge(merged_data, data, how='outer', on='region')
            
    cols = [col for col in merged_data.columns if 'Unnamed' not in col]
    
    return merged_data[cols]

In [29]:
file_paths = [os.path.join(os.getcwd(), 'rosstat', 'csv', file_name) for file_name in os.listdir(os.path.join(os.getcwd(), 'rosstat', 'csv'))]

merged_data = merge_data(file_paths)

In [30]:
print(merged_data.shape)

for region in not_inc_regions:
    merged_data = merged_data[merged_data['region'] != region].copy()
    
print(merged_data.shape)

merged_data.drop_duplicates('region', keep='last', inplace=True)

print(merged_data.shape)

(196761, 29)
(163920, 29)
(51, 29)


In [31]:
cols_with_nulls = merged_data.isnull().sum()
cols_with_nulls = cols_with_nulls[cols_with_nulls > 0].index

for col in cols_with_nulls:
    merged_data.fillna(merged_data[col].median(), inplace=True)
    
merged_data.reset_index(inplace=True, drop=True)

In [37]:
merged_data.to_csv('./data/data/add_data.csv', index=False)

In [32]:
train_data = pd.read_csv('./data/data/train.csv')
test_data = pd.read_csv('./data/data/test.csv')
train_data['region'] = train_data['region'].apply(lambda x: x.lower())

C:\Users\tuman\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
new_train_data = pd.merge(train_data, merged_data, on='region', how='left')

In [35]:
new_train_data.isnull().sum().sort_values()

city                                                  0
Доля городского населения в общей численности         0
price_type                                            0
realty_type                                           0
date                                                  0
                                                  ...  
reform_mean_floor_count_1000                      16708
reform_house_population_500                       27234
reform_mean_year_building_500                     29637
reform_mean_floor_count_500                       30168
floor                                            176237
Length: 105, dtype: int64